# Data Collection and Data Pre-processing

## Data Collection: 

    First, we need to collect the poems that will be the source for our project. To do this we will create a web-scraper in Python, that gather the poems of a specific writer from "mypoeticside.com" and save them to a .csv file.

In [1]:
# Import required libraries:

import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# This tricks the server to think that we are connecting from a web browser:

class AppURLopener(urllib.request.FancyURLopener): 
    version = "Mozilla/5.0" 
opener = AppURLopener()
writer = "william-shakespeare-poems"
data = opener.open('https://mypoeticside.com/poets/' + writer).read().decode()

<ipython-input-2-b8a58561f1f0>:5: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


In [3]:
# Search and save the poem links:

soup =  BeautifulSoup(data, 'html.parser')
poem_list = soup.find(class_="list-poems")
links = poem_list.findAll('a')
results = ["https:"+link.get('href') for link in links]

In [4]:
# Save the title and content of each poem:

titles = []
corpus = []
for page in results:
     data = opener.open(page).read().decode()
     soup = BeautifulSoup(data, 'html.parser')
     title = soup.find(class_='title-poem')
     poem = soup.find(class_='poem-entry')
     titles.append(title.getText())
     print(title.getText())
     corpus.append(poem.find('p').getText())

A Lover's Complaint
All the World's a Stage
Aubade
Blow, Blow, Thou Winter Wind
Bridal Song
Carpe Diem
Dirge
Dirge of the Three Queens
Fairy Land I
Fairy Land II
Fairy Land III
Fairy Land IV
Fairy Land v
Fidele
from Venus and Adonis
From you have I been absent in the spring... (Sonnet 98)
Orpheus with his Lute Made Trees
Sigh No More
Sonnet 106: When in the Chronicle of Wasted Time
Sonnet 8
Sonnet CVII: Not Mine Own Fears, Nor the Prophetic Soul
Sonnet CXI: O, for my sake do you with Fortune chide
Sonnet CXLV
Sonnet CXVI: Let me not to the marriage of true minds
Sonnet CXXX: My mistress' eyes are nothing like the sun
Sonnet I: From Fairest Creatures We Desire Increase
Sonnet II: When Forty Winters Shall Besiege Thy Brow
Sonnet III: Look In Thy Glass, and Tell the Face Thou Viewest
Sonnet IV: Unthrifty Loveliness, Why Dost Thou Spend
Sonnet LXIV: When I Have Seen by Time's Fell Hand Defac'd
Sonnet V: Those Hours, That With Gentle Work Did Frame
Sonnet XCIV: They That Have Power to Hurt 

In [5]:
# Save to .csv files for all the poems:

poems = pd.DataFrame({'title' : titles,'text' : corpus})
poems.to_csv('poems.csv')

## Data Pre-processing: 

    We create a function that takes our previously created .csv file and splits the poems into verses, based on a regular expression that we choose. Then, we also erased and replaced some undesired characters from the verses (colons, for example) in order to make the resulting poem looks more well connected and cohesive. We then saved the verses in a new .csv file.

In [6]:
import os
import pandas as pd
import numpy as  np
import re
import spacy

In [7]:
# Function that splits the poems in sentences, cleans them and saves them to a *.csv file:

def docs_to_sentences(file, split=r"\n"):
    
    path = os.getcwd()
    df_docs = pd.read_csv(path+"/" + file)
    number_docs = df_docs.shape[0]
    df_sentences = pd.DataFrame(columns=['doc_id','sentence'])
    
    for i in range(number_docs):
        
        text = df_docs.text[i]
        
        # Dictionary to replace unwanted elements:
        
        replace_dict = {'?«' :  '«', '(' :  '', ')' : '', ':' : ',', '.' : ',', ',,,' : ','}
        for x,y in replace_dict.items():
            text = text.replace(x, y)
        text = text.lower()
        
        # Split into sentences:
        
        sentences = re.split(split, text)
        len_sentences = len(sentences)   
        doc_id = [i] * (len_sentences)
        
        # Save sentence and poem_id:
        
        doc_sentences = pd.DataFrame({'doc_id' : doc_id, 'sentence' : sentences})
        df_sentences = df_sentences.append(doc_sentences)   
        
    # Extra cleaning and reset index:
    
    df_sentences = df_sentences[df_sentences.sentence != '']
    df_sentences.reset_index(drop=True, inplace=True)  
    
    # Save clean sentences to a .csv file: 
    
    df_sentences.to_csv("sentences_" + file)

In [8]:
# Saves sentences to sentences_poems.csv file:

df = docs_to_sentences(file='poems.csv', split=r"\n")

In [9]:
import os
import spacy
import pandas as pd
import numpy as np

path = os.getcwd()
file='sentences_poems.csv'
sentences = pd.read_csv(path+'/'+ file).fillna("")

In [10]:
sentences['sentence'].replace('', np.nan, inplace=True)

In [11]:
sentences.dropna(subset=['sentence'], inplace=True)

In [12]:
sentences.head(10)

,Unnamed: 0,doc_id,sentence
0,0,0,from off a hill whose concave womb reworded
1,1,0,"a plaintful story from a sistering vale,\r"
2,2,0,my spirits to attend this double voice acc...
3,3,0,and down i laid to list the sad-tuned tale;
4,4,0,"ere long espied a fickle maid full pale,\r"
5,5,0,"tearing of papers, breaking rings a-twain,\r"
6,6,0,storming her world with sorrow's wind and ...
8,8,0,"upon her head a platted hive of straw,\r"
9,9,0,"which fortified her visage from the sun,\r"
10,10,0,whereon the thought might think sometime i...


In [13]:
sentences = sentences['sentence']

In [14]:
sentences

0             from off a hill whose concave womb reworded
1              a plaintful story from a sistering vale,\r
2           my spirits to attend this double voice acc...
3             and down i laid to list the sad-tuned tale;
4              ere long espied a fickle maid full pale,\r
                              ...                        
4957                     make the gruel thick and slab,\r
4958                    add thereto a tiger's chaudron,\r
4959                   for ingrediants of our cauldron,\r
4960                    double,double toil and trouble,\r
4961                       fire burn and cauldron bubble,
Name: sentence, Length: 4408, dtype: object

In [15]:
sentences = sentences.values.tolist()

In [16]:
sentences

['from off a hill whose concave womb reworded',
 '    a plaintful story from a sistering vale,\r',
 '    my spirits to attend this double voice accorded,\r',
 '    and down i laid to list the sad-tuned tale;',
 '    ere long espied a fickle maid full pale,\r',
 '    tearing of papers, breaking rings a-twain,\r',
 "    storming her world with sorrow's wind and rain,",
 '    upon her head a platted hive of straw,\r',
 '    which fortified her visage from the sun,\r',
 '    whereon the thought might think sometime it saw',
 '    the carcass of beauty spent and done,\r',
 '    time had not scythed all that youth begun,\r',
 "    nor youth all quit; but, spite of heaven's fell rage,\r",
 "    some beauty peep'd through lattice of sear'd age,",
 '    oft did she heave her napkin to her eyne,\r',
 '    which on it had conceited characters,\r',
 '    laundering the silken figures in the brine',
 "    that season'd woe had pelleted in tears,\r",
 '    and often reading what contents it bears;',

In [17]:
# using list comprehension + split() 
# Tokenizing strings in list of strings

tokenized_sentences = [sub.split() for sub in sentences] 

In [18]:
tokenized_sentences

[['from', 'off', 'a', 'hill', 'whose', 'concave', 'womb', 'reworded'],
 ['a', 'plaintful', 'story', 'from', 'a', 'sistering', 'vale,'],
 ['my', 'spirits', 'to', 'attend', 'this', 'double', 'voice', 'accorded,'],
 ['and', 'down', 'i', 'laid', 'to', 'list', 'the', 'sad-tuned', 'tale;'],
 ['ere', 'long', 'espied', 'a', 'fickle', 'maid', 'full', 'pale,'],
 ['tearing', 'of', 'papers,', 'breaking', 'rings', 'a-twain,'],
 ['storming', 'her', 'world', 'with', "sorrow's", 'wind', 'and', 'rain,'],
 ['upon', 'her', 'head', 'a', 'platted', 'hive', 'of', 'straw,'],
 ['which', 'fortified', 'her', 'visage', 'from', 'the', 'sun,'],
 ['whereon', 'the', 'thought', 'might', 'think', 'sometime', 'it', 'saw'],
 ['the', 'carcass', 'of', 'beauty', 'spent', 'and', 'done,'],
 ['time', 'had', 'not', 'scythed', 'all', 'that', 'youth', 'begun,'],
 ['nor',
  'youth',
  'all',
  'quit;',
  'but,',
  'spite',
  'of',
  "heaven's",
  'fell',
  'rage,'],
 ['some', 'beauty', "peep'd", 'through', 'lattice', 'of', "sear'

In [20]:
# Create reverse N-grams from a list of tokens:

def reverseNgrams(tokens, n):
    ngrams = []
    for i in range(len(tokens)-1, 0+n-2, -1):
        ngram = []
        for j in range(i, i-n, -1):
            ngram.append(tokens[j])
        ngrams.append(ngram)
    return ngrams

In [21]:
# Organize N-grams in a frequency lookup table (N-layer nested dictionaries):

def setupModel(ngrams):
    lookup = {}
    n = len(ngrams[0])
    for ngram in ngrams:
        ptr = lookup
        for i in range(0, n):
            if i == n-1:
                ptr.setdefault(ngram[i], 0)
                ptr[ngram[i]] += 1
            else:
                try: ptr = ptr[ngram[i]]
                except KeyError: 
                    ptr.setdefault(ngram[i], {})
                    ptr = ptr[ngram[i]]
    return lookup

In [22]:
# Process text and collect reverse N-grams sentence by sentence
# Do not do this word by word or you'll have incoherent N-grams that span sentences

ngrams = []
for sentence in tokenized_sentences:
    ngrams += reverseNgrams(sentence, 3)

In [23]:
ngrams

[['reworded', 'womb', 'concave'],
 ['womb', 'concave', 'whose'],
 ['concave', 'whose', 'hill'],
 ['whose', 'hill', 'a'],
 ['hill', 'a', 'off'],
 ['a', 'off', 'from'],
 ['vale,', 'sistering', 'a'],
 ['sistering', 'a', 'from'],
 ['a', 'from', 'story'],
 ['from', 'story', 'plaintful'],
 ['story', 'plaintful', 'a'],
 ['accorded,', 'voice', 'double'],
 ['voice', 'double', 'this'],
 ['double', 'this', 'attend'],
 ['this', 'attend', 'to'],
 ['attend', 'to', 'spirits'],
 ['to', 'spirits', 'my'],
 ['tale;', 'sad-tuned', 'the'],
 ['sad-tuned', 'the', 'list'],
 ['the', 'list', 'to'],
 ['list', 'to', 'laid'],
 ['to', 'laid', 'i'],
 ['laid', 'i', 'down'],
 ['i', 'down', 'and'],
 ['pale,', 'full', 'maid'],
 ['full', 'maid', 'fickle'],
 ['maid', 'fickle', 'a'],
 ['fickle', 'a', 'espied'],
 ['a', 'espied', 'long'],
 ['espied', 'long', 'ere'],
 ['a-twain,', 'rings', 'breaking'],
 ['rings', 'breaking', 'papers,'],
 ['breaking', 'papers,', 'of'],
 ['papers,', 'of', 'tearing'],
 ['rain,', 'and', 'wind'],


In [25]:
# Organize N-grams in a frequency lookup table (N-layer nested dictionaries):

model = setupModel(ngrams)

In [26]:
for row in model:
    print(row, model[row])

reworded {'womb': {'concave': 1}}
womb {'concave': {'whose': 1}, 'hollow': {'whose': 2}, "unear'd": {'whose': 1}, 'dark': {"earth's": 1}, "puddle's": {'a': 1}}
concave {'whose': {'hill': 1}}
whose {'hill': {'a': 1}, 'one': {'not': 1}, 'in': {'emerald,': 1}, 'fair': {'so': 1}, 'dear,': {'is': 1}, 'me,': {'to': 1}, 'her': {'darken': 1}, 'coverlet;': {'green': 1}, 'they': {'than': 1}, 'breath,': {'celestial': 1}, 'love,': {'in': 1}, 'sickness,': {'marrow-eating': 1}, 'theirs': {'or': 1}, 'from': {'morning,': 2}, 'doe,': {'milch': 2}, 'snail,': {'the': 2}, 'flank;': {'soft': 2}, 'by': {'doves;': 1}}
hill {'a': {'off': 1}}
a {'off': {'from': 1}, 'from': {'story': 1, 'favours': 1, 'me': 1, 'running': 1, 'as': 3, "fortress'd": 1, 'coming': 1}, 'espied': {'long': 1}, 'head': {'her': 1}, 'her': {'in': 1, 'bear': 2}, 'in': {'she': 3, 'falseness': 1, 'spirit': 1, 'and': 2, 'which': 1, 'confounded': 1, 'pleads,': 1, 'deep-drenched': 1, 'who': 1, 'as': 1, 'pouted': 1, 'tangled': 1, 'prisoner': 1, '

much {'so': {'and': 1, 'heart': 1, 'is': 2}, 'how': {'by': 1}, 'too': {'something': 1, 'with': 2, 'and': 1}, 'that': {'those': 1}, 'as': {"assay'd": 1}, 'and': {'mischances': 1}, "'how": {'she,': 1}, '"how': {'she,': 1}}
contains, {'reproach': {'their': 1}}
reproach {'their': {'me': 1}, "black'st": {'thou': 1}, 'my': {'couple': 1}, 'undeserved': {'and': 1}}
seen, {'have': {'eyes': 1}, 'not': {'art': 1}, 'was': {'face': 1}, 'footing': {'no': 1}, 'had': {'more': 1}, 'be': {'not': 1}}
warm'd, {'as': {'much': 1}}
heart {'my': {'flame': 1, 'gave': 1, 'persuade': 1, 'against': 1, 'buys': 1, 'know,': 1, 'no;': 1, 'shame,': 1, 'fire,': 1}, 'rocky': {'what': 1}, 'a': {'not': 1, 'where': 1}, 'his': {'from': 1, 'is': 1, 'but': 1}, 'of': {'grief': 1}, 'her': {'in': 1, 'wakes': 1}, 'the': {'in': 2, 'breast,': 1, 'say,': 1, 'set': 1, 'wound': 2}, 'false': {'my': 1}, 'drumming': {'his': 1}, 'pure': {'a': 1}, 'own': {'thine': 1}, 'thy': {'thine,': 1}, 'hard': {'thy': 1, 'that': 1}, "adonis'": {'becaus

gone, {'being': {'spines': 1, 'and,': 1}, 'art': {'home': 1}, 'be': {'to': 2, 'i': 1, 'horse': 1}, 'quite': {'leaves': 1}, 'is': {'horse': 1}, 'tears': {'salt': 1}}
spines {'sharp': {'their': 1}}
alone, {'smells': {'their': 1}, 'self': {'thy': 1}, 'love': {'my': 1}, 'all': {'her': 1}, 'eye': {'the': 1}, 'sleeps': {'it': 1}}
smells {'their': {'in': 1}}
hue; {'their': {'in': 1}}
faint, {'odour': {'of': 1}, 'so': {'is': 1}, 'and': {'pale': 1}}
odour {'of': {'pinks,': 1}}
quaint, {'most': {'yet': 1}}
true; {'thyme': {'sweet': 1}, "'tis": {'true,': 1}}
thyme {'sweet': {'and': 1}}
ver; {'of': {'child': 1}}
child {'firstborn': {'primrose,': 1}, 'and': {'sire': 1}, 'fair': {"'this": 1}, 'the': {'make': 1}}
harbinger, {"springtime's": {'merry': 1}, 'shrieking': {'thou': 1}}
dim; {'bells': {'her': 1}}
bells {'her': {'with': 1}}
growing, {'cradles': {'their': 1}}
cradles {'their': {'in': 1}}
blowing, {'death-beds': {'on': 1}}
death-beds {'on': {'marigolds': 1}}
children {"nature's": {'dear': 1}, 

day {'gentle': {'as': 1}, 'your': {'change': 2}, 'of': {'break': 2}, 'such': {'of': 1}, 'jealous': {'the': 1}, "summer's": {'a': 1}, 'the': {'shall': 1, 'dead,': 1}, 'by': {'sun': 1}, 'thee': {'rock': 1}}
flown {'is': {'hell': 1}}
away {'hate': {'from': 1}, 'breath,': {'fetched': 1}, 'drives': {'and': 1}, 'lure,': {'the': 2}, 'world,': {'the': 1}}
you' {"'not": {'saying': 1}}
'not {'saying': {'life,': 1}}
saying {'life,': {'my': 1}}
minds {'true': {'of': 2}, "men's": {'doth': 2}}
marriage {'the': {'to': 2}}
finds, {'alteration': {'it': 2}}
alteration {'it': {'when': 2}}
remover {'the': {'with': 2}}
mark {'ever-fixed': {'an': 2}, 'scornful': {'the': 1}, 'to': {'time': 1}, 'true': {'tongue,': 1}, 'not': {'thou': 1}}
ever-fixed {'an': {'is': 2}}
shaken; {'never': {'is': 2}}
tempests {'on': {'looks': 2}}
wandering {'every': {'to': 1}, 'a': {'hive': 1}}
star {'the': {'is': 2}, 'comfortable': {'no': 1}, 'bright': {'a': 1}, 'shining': {'and': 2}}
taken, {'be': {'height': 2}}
height {'his': {'

turn'd {'be': {'angel': 1}, 'is': {'now': 1}, 'are': {'eyes': 1}}
tell; {'directly': {'not': 1}}
directly {'not': {'but': 1}}
another's {'in': {'angel': 1}}
out, {'one': {'good': 1}, 'is': {'heaven': 1}, 'owner': {'the': 1}, 'blown': {'soon': 1}, 'burnt': {'lamps,': 1}}
bad {'my': {'till': 1}, 'to': {'that': 1}, 'so': {'still': 1}}
rhetoric {'heavenly': {'the': 1}}
argu- {'hold': {'not': 1}}
hold {'not': {'could': 1}, 'and': {'bones,': 1}, 'ground?': {'the': 1}, 'will': {'world': 1}}
perjury? {'false': {'this': 1}}
punishment, {'not': {'deserve': 1}}
deserve {'broke': {'thee': 1}}
broke {'thee': {'for': 1}}
forswore; {'i': {'woman': 1}}
forswore {'i': {'goddess,': 1}}
goddess, {'a': {'being': 1}}
love; {'heavenly': {'a': 1}, 'her': {'beg': 1}, 'my': {'to': 1}, 'but': {'choose': 1}, 'to': {'learn': 1}}
earthly, {'was': {'vow': 1}}
cures {"gain'd": {'being': 1}}
gain'd {'being': {'grace': 1}}
is; {'vapour': {'a': 1}, 'it': {"'shameful": 1, 'hateful': 1}}
vapour {'a': {'breath': 1, 'like'

...

imagination! {'still': {'in': 1}}
receipt, {'his': {'vomit': 1}}
vomit {'must': {'desire': 1}}
abomination, {'own': {'his': 1}}
exclamation {'no': {'pride,': 1}}
rein {'or': {'heat': 1}}
tire, {'doth': {'himself': 1}}
self-will {'jade': {'a': 1}}
jade {'a': {'like': 2}}
cheek, {"discolour'd": {'lean': 1}, 'his': {'stroke': 1, 'brow,': 1}, 'pale': {'her': 1}}
discolour'd {'lean': {'and': 1}}
lank {'with': {'then': 1}}
pace, {'strengthless': {'and': 1}}
strengthless {'and': {'brow,': 1}}
knit {'eye,': {'heavy': 1}}
meek, {'and': {'poor,': 1}}
poor, {'recreant,': {'all': 1}}
recreant, {'all': {'desire,': 1}}
wails {'beggar': {'bankrupt': 1}}
decays, {'that': {'when': 1}}
revels; {'it': {'there': 1}}
prays, {'remission': {'for': 1}}
remission {'for': {'rebel': 1}}
faultful {'this': {'with': 1}}
chased; {'hotly': {'so': 1}}
hotly {'so': {'accomplishment': 1}, 'did': {'eye': 1}}
accomplishment {'this': {'who': 1}}
disgraced, {'stands': {'he': 1}}
defaced; {'is': {'temple': 1}}
temple {'fair'

servants {'words,': {'idle': 1}}
words, {'idle': {"'out,": 1}, 'fair': {'speak': 1}}
arbitrators! {'weak': {'sounds,': 1}}
schools; {'skill-contending': {'in': 1}}
skill-contending {'in': {'yourselves': 1}}
debaters; {'dull': {'with': 1}}
serves {'leisure': {'where': 1}}
leisure {'where': {'debate': 1}}
mediators, {'you': {'be': 1}}
clients {'trembling': {'to': 1}}
argument {'not': {'force': 1}}
case {'my': {'that': 1}}
rail {'i': {'vain': 1}}
uncheerful {'and': {'tarquin,': 1}}
tarquin, {'at': {'time,': 1}}
infamy, {'mine': {'with': 1}, 'living': {'to': 1}, 'daring': {'with': 1}}
cavil {'i': {'vain': 1}}
despite, {"confirm'd": {'my': 1}, "heaven's": {'high': 1}}
confirm'd {'my': {'at': 1}}
spurn {'i': {'vain': 1}}
indeed {'remedy': {'the': 1}}
foul-defiled {'my': {'forth': 1}}
decree? {'this': {'at': 1}}
quiver'st {'why': {'hand,': 1}}
rid {'to': {'thyself': 1}}
livest {'thou': {'live,': 1}}
defend {'not': {'couldst': 1}, 'well': {'so': 1}}
wicked {'her': {'scratch': 1}}
scratch {'to'

dew'd {'were': {'they': 1}}
net, {'a': {'in': 1}}
tangled {'lies': {'bird': 1}}
awed {'and': {'shame': 1}}
bank, {'the': {'overflow': 1}}
overflow {'it': {'force': 1}}
entreats, {'prettily': {'and': 1}, 'she': {'still': 1}}
prettily {'and': {'entreats,': 1}}
frets, {'and': {'lours': 1}, 'treasure': {'hidden': 1}}
lours {'he': {'still': 1}}
sullen, {'he': {'is': 1}}
ashy-pale, {'anger': {'and': 1}}
best; {'him': {'loves': 1}}
better'd {'is': {'best': 1}}
swears, {'she': {'hand': 1}}
immortal {'fair': {'her': 1}}
truce {'take': {'he': 1}}
wet; {'all': {'cheeks': 1}}
rain'd, {'have': {'long': 1}}
countless {'this': {'pay': 1}}
raise {'he': {'did': 1}}
promise {'this': {'upon': 1}}
wave, {'a': {'through': 1}}
peering {'dive-dapper': {'a': 1}}
dive-dapper {'a': {'like': 1}}
in; {'quickly': {'as': 1}}
quickly {'as': {'ducks': 1}, 'and': {'told': 1}, 'not': {'they': 1}, 'chariot': {'light': 1}}
ducks {'on,': {"look'd": 1}}
on, {"look'd": {'being': 1}, 'bonnet': {'his': 1}}
pay, {'his': {'for'

gone? {'quickly': {'and': 1}}
double, {'should': {'debt': 1}}
non-payment {'for': {'say,': 1}}
trouble? {'a': {'such': 1}}
'if {'he,': {'quoth': 1}}
years, {'unripe': {'my': 1}, 'tender': {'his': 1}}
strangeness {'my': {'measure': 1}}
forbears, {'fry': {'ungrown': 1}}
fry {'ungrown': {'the': 1}}
ungrown {'the': {'but': 1}}
sticks {'green': {'the': 1}}
gait, {'weary': {'with': 1}}
comforter, {"world's": {'the': 1}}
west; {'the': {'in': 1}}
late;' {'very': {"''tis": 1}}
''tis {'shrieks,': {'herald,': 1}, 'all': {'answer': 1}, 'says': {'she': 1}}
shrieks, {'herald,': {"night's": 1}}
herald, {"night's": {'owl,': 1}}
'good {'say': {'me': 1}}
kiss,' {'a': {'have': 1}}
'adieu,' {'says': {'he': 1}}
tender'd {'parting': {'of': 1}}
fee {'honey': {'the': 1}}
embrace; {'sweet': {'a': 1}}
seem; {'they': {'then': 1}}
backward {'and': {"disjoin'd,": 1}}
disjoin'd, {'he': {'breathless,': 1}}
thirsty {'her': {'taste': 1}}
drouth, {'on': {'complain': 1}}
dearth {'with': {'faint': 1}, 'barren': {'and': 1

spread, {'sinews': {'her': 2}}
sinews {'her': {'all': 2}}
whither, {'not': {'knows': 2}}
hurries {'madly': {'which': 2}}
murther, {'for': {'boar': 2}}
ways; {'thousand': {'a': 2}}
spleens {'thousand': {'a': 2}}
again; {'untreads': {'she': 2}, 'forth': {'creep': 2}, 'looks': {'with': 2}}
untreads {'she': {'that': 2}}
path {'the': {'treads': 2}}
mated {'is': {'haste': 2}}
brain, {'drunken': {'a': 2}}
drunken {'a': {'of': 2}, 'but': {'is': 2}}
respecting; {'all': {'at': 2}}
respects, {'of': {'full': 2}}
hound, {'a': {'finds': 2}}
caitiff {'weary': {'the': 2}}
wound, {'his': {'of': 2}}
licking {'another': {'there': 2}}
plaster; {'sovereign': {'only': 2}}
sores {"venom'd": {"'gainst": 2}}
scowling, {'sadly': {'another': 2}}
meets {'she': {'here': 2}}
howling, {'with': {'replies': 2}}
replies {'he': {'and': 2}}
noise, {'ill-resounding': {'his': 2}}
ill-resounding {'his': {'ceased': 1, "ceas'd": 1}}
ceased {'hath': {'he': 1}}
grim, {'and': {'black': 2}, 'foul,': {'this': 1}}
mourner, {"flap-m

In [27]:
import json

with open('all-poems.json', 'w') as outfile:
    json.dump(model, outfile)

------------------------------------------------------------------------------------------------------------------